# The RLE-FED Algorithm 
by Man Hei (Thomas) Cheng @ Imperial College London

## Introduction: 

In this notebook, we will demonstrate the use case of the Randomized Linear Encoder (RLE) and Fermionic Expectation Decoder (FED).

On top of that, we also provide a very concise code for plane molecule generation using PySCF, and exact diagonalisation.

## $\text{LiH}$ demo: 
Following is the illustrative compression of the LiH molecule $8$ modes compressed to $6$ qubits: 

### The RLE algorithm:

RLE begins its random search by choosing a $Q$ within the bounded region $N\log_2 M < Q < 2N\log_2 M$, with $M$ the fermionic modes and $N$ the number of electrons. Then, the algorithm initializes the parity check matrix $\mathbf{G} = [I_Q | D]$, where $I_Q$ is a $Q \times Q$ identity matrix. The form of $\mathbf{G}$ is known as the standard basis and it helps reduce the space of randomized search. We randomly generate the $Q \times (M-Q)$ matrix $D$ such that each column has even Hamming weight equal to the value $Q/2$, denoted as $\text{even}(Q/2)$.

$$
\vec{k} = \mathbf{C}\vec{v}, \vec{v}\in S
$$

where $S$ is defined as the set of all vectors with Hamming weight $2$ and $2K$ for $2K<N$, where $K$ is a randomly generated integer. The random check serves to maximize the probability of obtaining the correct encoder. Finally, the algorithm explicitly computes the states to check for repetitions. This process is repeated until the minimum $Q$ is found.

In [1]:
import numpy as np
import diag as d
import RLE as r
import FED as fed
#generate LiH molecule
dist = np.array([2.5])
atoms = ['Li', 'H']
Rs = np.array([1, 0])
thetas = [0, 0]
basis = 'sto-3g'
mult = 1

# Activate orbitals (ACM, *right?*)
orb_choice = [1, 2, 3, 5]

#optional fermionic to hardcore bosonic encoding
hardcore = 0

mols = []
for i in dist:
    R = i*Rs
    mols.append(d.plane_molecule(atoms, R, thetas, basis, mult))


#The number of selected fermionic modes
M = len(orb_choice)*2

#The number of electrons
N = mols[0].nelectron-2*orb_choice[0]

#The encoded qubits
Q = 6

#Ground State Energy -- optional
basis = d.new_hot(mols[0], orb_choice[0], orb_choice, hardcore)
functions = [[d.compute_integrals, d.get_active_space_integrals, basis]]

#get energy, ground state and the optional meanfield object.
ene, gs, MF= d.spectrum(mols, functions, orb_choice[0], orb_choice)
print('gs: ', ene[0][0])

#generate encoder
B = r.encoder(M, N, Q)

#generate look-up-table decoder
tableb = fed.look_up_parity(M, 2*N+1, N, B)

#Store decoding information as dictionary.
print(tableb)

#we try this one first.

converged SCF energy = -7.7708736692219
gs:  -7.823254150880832
0 0
Injectivity:  28 28 1.0
{'111100': '00000011', '011001': '00000101', '100101': '00000110', '011010': '00001001', '100110': '00001010', '000011': '00001100', '011100': '00010001', '100000': '00010010', '000101': '00010100', '000110': '00011000', '010000': '00100001', '101100': '00100010', '001001': '00100100', '001010': '00101000', '001100': '00110000', '001000': '01000001', '110100': '01000010', '010001': '01000100', '010010': '01001000', '010100': '01010000', '011000': '01100000', '111000': '10000001', '000100': '10000010', '100001': '10000100', '100010': '10001000', '100100': '10010000', '101000': '10100000', '110000': '11000000', '000000': '00000000', '000001': '00000100', '000010': '00001000', '111101': '00000111', '111110': '00001011', '011011': '00001101', '100111': '00001110', '011101': '00010101', '011110': '00011001', '000111': '00011100', '101101': '00100110', '101110': '00101010', '001011': '00101100', '0011

#### Remark: 

The RLE algorithm is intrinsically non-scalable with respect to the number of electrons -- namely $O(M^N)$ with $M$ the number of fermionic modes and $N$ the number of electrons. Its associated decoder -- the look-up-table decoder -- is also not scalable. However, this problem can be circumvented with the Gallager's LDPC encoder-decoder where the corresponding encoder-decoder process are performed in polynomial time.

### The FED algorithm:

Next, we demonstrate how to decode the energy expectation value with the FED algorithm. The FED itself is encoder-decoder agnostic, which means that it can adopt any existing encoding-decoding scheme to fermionic simulation.

What follows is the workflow of the FED algorithm. First, we introduce the basic variables required to compute the fermionic expectation value under our FED algorithm. First, we need the first, second electronic integrals with the frozen core energy given by `integrals`. From `integrals`, we compute `base` which contains the indices of the non-zero RDM elements, which we will measure in the quantum computer. From `base`, we obtain the `groups` data. `groups` contains information of all distinct measurement bases prepared for the quantum experiment, and all RDM operators which require measurement. 

We note that although `compute` contains redundant RDM terms, it is necessary for computing the correct energy because the numerical first and second electronic integrals do not exactly follow the integral symmetries. That is, for example $g_{ijkl}\sim g_{jikl}$. Meanwhile, from the `groups` object, which contains the necessary RDMs to measure on the quantum computer, we can compute the information of the measurement basis from `B` and `ustrings`, the anti-symmetry sign of each RDM values under the fermionic-to-qubit mapping `signs = fed.glob_sign(groups, M)`.

Finally, after the quantum experiment, we typically obtain a set of histograms `hists`, from which we can finally calculate the RDM expectation values $\langle \hat{a}^\dagger_i\hat{a}^\dagger_j\hat{a}_k\hat{a}_l\rangle$ and $\langle\hat{a}^\dagger_i\hat{a}_j\rangle$ of a quantum state $|\psi\rangle$, using the FED algorithm `fed.rdm_val`. The energy expectation is then given by the  energy formula $\sum_{ij}h_{ij}\langle\hat{a}^\dagger_i\hat{a}_j\rangle+\frac{1}{2}\sum_{ijkl}g_{ijkl}\langle \hat{a}^\dagger_i\hat{a}^\dagger_j\hat{a}_k\hat{a}_l\rangle$.

In this example, we benchmark the energy function with the ground state energy of LiH solved from exact diagonalisation previously with the function `fed.hist_format`. This function computes the histograms obtained from different measurement bases of a real time quantum computer generated by `groups = fed.measurement_group(compute)`. Meanwhile, the `fed.rdm_val` takes the look up table decoder `tableb` generated from the previous section for decoding the fermionic information from the histograms `hists`. 

While in our demonstration the use of `tableb` decoder requires $\mathcal{O}(M^N)$ memory, in practice this can be immediately (ACM, *?*) with a LDPC decoder provided that the encoder `B` is an LDPC matrix.

In [2]:
'compute integrals and PySCF meanfield object'
stuff, MF= d.integrals(mols, functions, orb_choice)

naos = np.arange(mols[0].nao)

#calculate all non-zero contributions of the RDM, neglecting RDM symmetries
base = fed.Indices(stuff[0], threshold = 10**-12)

#compute groups (ACM, *right?*)
compute = fed.RDM_grouper(base)
#determines the measurement groups
groups = fed.measurement_group(compute)

#compute x-string
ustrings = fed.xfunc(groups, np.identity(M))
signs = fed.glob_sign(groups, M)

#Generate histrogram from ground state -- to mimick what we measured from quantum computer
hists = fed.hist_format(gs[0][0], basis, B, ustrings)

#From the histogram, we calculate the rdm values
da, ddaa, Vals= fed.rdm_val(hists, tableb, groups, signs, B, hardcore)

one = stuff[0][0]
two = stuff[0][1]
core = stuff[0][2]

print('from partial RDM: ', np.tensordot(one, da, axes=((0,1), (0,1)))+1/2*np.tensordot(two, ddaa, axes=((0,1,2,3), (0,1,2,3)))+MF[0].energy_nuc()+core)

converged SCF energy = -7.77087366922191
from partial RDM:  -7.823254150880836


#### Remark:
With the energy function in hand, one can design the cost function for a VQE simulation. The routine of the cost function involves histogram sampling and decoding with the FED algorithm for the expectation values. In the LiH example, the number of measurement bases is $25$. It means that each query of the energy function requires $25$ measurements. Compared to non-linear compression, $25$ is a significantly more managable number. 

To ensure the correctness of this protocol, we also benchmark it against the actual RDM values as follow:

In [3]:
'Double check another one.'
ada = d.RDM1(gs[0][0], basis)

ddaa = d.RDM2(gs[0][0], basis)


'need to compute the integrals.'
stuff, MF = d.integrals(mols, functions, orb_choice)

one = stuff[0][0]
two = stuff[0][1]
core = stuff[0][2]


print('from RDM: ', np.tensordot(one, ada, axes=((0,1), (0,1)))+1/2*np.tensordot(two, ddaa, axes=((0,1,2,3), (0,1,2,3)))+MF[0].energy_nuc()+core)


for i in zip(groups, Vals):
    for index, j in enumerate(i[0]):
        if len(j) == 2:
            
            if np.abs((np.abs(da[j[0], j[1]])-np.abs(i[1][index])))>1e-15:
                print('1-RDM: ', j, ada[j[0], j[1]], i[1][index])
                
        if len(j) == 4:
            if np.abs((np.abs(ddaa[j[0], j[1], j[2], j[3]])-np.abs(i[1][index])))>1e-10:
                print('2-RDM: ', j, ddaa[j[0], j[1], j[2], j[3]], i[1][index])


converged SCF energy = -7.77087366922191
from RDM:  -7.823254150880832


#### Remark:

From the FED algorithm, we can efficiently decode the partial 1-RDM and 2-RDM terms `da` and `ddaa` which are responsible for the energy contribution. Consequently, one can design algorithms, such as VQE, which makes use of the RDMs observables. For example, in our benchmark we design the cost function calculator. 

For specification, we use qiskit 0.45.1 for demonstration. In what follow, we provide an example code for the energy cost function for $\text{LiH}$ using a noiseless Aer sampler. 

### Cost Function:

With the decoder able to correctly decode the histogram generated from the ground state, we develop the `circuit_energy` function which calculates the fermionic Hamiltonian energy by decoding the basis of the encoded operator $|{\vec{a}}\rangle\langle{\vec{b}}|+|{\vec{b}}\rangle\langle{\vec{a}}|$ instead of Pauli decomposition which is deemed unscalable. We provide an example of energy  

In [4]:
import numpy as np
from qiskit_aer import AerProvider
from qiskit_aer.backends import AerSimulator
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit.circuit import ParameterVector

In [5]:
#prepare backend
backend = AerProvider().backends()[1]
print(AerProvider().backends())
backend = AerSimulator(method = 'density_matrix')
backend.shots = 100000

def sample(backend, circuit, layout, correction, count = 0):
    '''
    sample quantum circuit state distribution
    
    Parameters
    ----------
    circuit: qiskit circuit (no params)
    
    Returns
    -------
    dictionary of probability distribution
    
    '''
    val = backend.shots
    job = backend.run(circuit, shots = int(val))
    counts = job.result().get_counts()
    # evaluate probabilities
    shots = sum(counts.values())
    prob = {b: c/shots for b, c in counts.items()}    
    return prob 

def clifford_basis(circuit, xstrings):
    '''
    Generate measurement basis
    '''

    circuits = []
    
    for xstring in xstrings:
        cop = circuit.copy()
        count = 0
        cnot = []
        h = []
        for index, i in enumerate(xstring):
            if (i == 1) and count == 0:
                mem = index
                count=1
                h.append(index)
            elif (i == 1) and count:
                #print(mem, index)
                cnot.append((mem, index))
                mem = index
        cnot = cnot[::-1]
        for (i,j) in cnot:
            cop.cx(i, j)
        for i in h:
            cop.h(i)
        cop.measure_all()
        circuits.append(cop)
    return circuits

def circuit_energy(params, mf, inte, backend, circuit, xstrings, table, groups, signs, A, save = 0):
    '''
    Circuit energy only via the sampling means.

    Input:
    ------
    mf: PySCF meanfield object.

    inte: core energy plus electronic integrals
    
    circuit: qiskit circuit.

    params: the parameters to combine. 

    xstrings: the encoded Pauli-x strings for the preparation of Clifford measurement basis.

    table: look-up-table decoder replacable by classical decoder.    

    groups: measurement groups for decoding.

    signs: global fermionic signs 

    Output:
    -------
    energy: the energy
    '''

    'First step: you will just need circuit and assign_parameters'
    
    circuit = circuit.assign_parameters(params)

    
    circuits = clifford_basis(circuit, xstrings)

    'Those circuits could be wrong.'


    'In here, you must sample each of them, where we will need our backend information.'

    'prioritize sampling'
    hists = []
    data = []
    'check one stats'
    #count = 0

    'This part takes most of the time.'
    for circuit in circuits:
        stats = sample(backend, circuit, None, correction = 0, count = 0)
        keys = []
        
        for i in stats.keys():
            keys.append(i[::-1])
        stats = dict(map(lambda i,j : (i,j) , keys,stats.values()))

        #if count == 0:
            #stats = dict(sorted(stats.items(), key=lambda item: toint(item[0])))
            #example.append(stats)
                    
        'save your data'
        if save == 1:
            save = data_stuff()
            save.write_dict('stats', stats)

        hists.append(stats)
        data.append(save)
        
    'we need to reverse the histogram I think.'
    
    da, ddaa, Vals = fed.rdm_val(hists, table, groups, signs, A, 0)

    
    one = inte[0]
    two = inte[1]
    core = inte[2]
    e = np.tensordot(one, da, axes=((0,1), (0,1)))+1/2*np.tensordot(two, ddaa, axes=((0,1,2,3), (0,1,2,3)))+MF[0].energy_nuc()+core
    return e


def makecircuit(init, cnot, reps, name = 'gg', HF = 0, sing = 1):

    '''
    Make your hardware efficient quantum circuit.
    
    Input:
    ------
    orb: orbital number of your choice

    e: electron number after freeze

    A: encoder

    cnot: your onfiguration of cnot layer

    rep: the number of circuit repetition
    '''
    #that is the problem. 

    n=init.num_qubits
    param = ParameterVector(name,int(n*reps)+n*sing)
    t=0
    for l in range(reps):
        for i in range(n):
            init.ry(param[t],i)
            t+=1
        for (i, j) in cnot:
            init.cx(i,j)
        init.barrier()
    if sing == 1:
        for i in range(n):
            init.ry(param[t],i)  
            t+=1
    return init     


HF = np.array([1, 1, 0, 0, 0, 0, 0, 0])
HF = B@HF

odd = []
even = []
n = 6

'One layer for demonstration'
reps = 0
for i in range(n//2):
    'odd'
    odd.append((2*i, 2*i+1))
    even.append((2*i+1, 2*i+2))
if n%2 == 0:
    even.pop()
cnot = odd+even


qr = QuantumRegister(len(HF), name = 'hea')
init = QuantumCircuit(qr)
init.x(0)
init.x(1)
cir = makecircuit(init, cnot, reps)

print(cir)

params = np.random.normal(0, 2*np.pi, len(cir.parameters))*0
inte, MF = d.integrals(mols, functions, orb_choice)

xstrings = fed.xfunc(groups, B)

'circuit energy by sampling -- test Hartree-Fock energy to show that it matches'
print('sampled HF energy: ', circuit_energy(params, MF[0], inte[0], backend, cir,  xstrings, tableb, groups, signs, B))


HF = np.array([1, 1, 0, 0, 0, 0, 0, 0])
HF = B@HF

odd = []
even = []
n = 6

'One layer for demonstration'
reps = 3
for i in range(n//2):
    'odd'
    odd.append((2*i, 2*i+1))
    even.append((2*i+1, 2*i+2))
if n%2 == 0:
    even.pop()
cnot = odd+even


qr = QuantumRegister(len(HF), name = 'hea')
init = QuantumCircuit(qr)
init.x(0)
init.x(1)
cir = makecircuit(init, cnot, reps)

print(cir)

params = np.random.normal(0, 2*np.pi, len(cir.parameters))
inte, MF = d.integrals(mols, functions, orb_choice)

xstrings = fed.xfunc(groups, B)

'benchmark circuit energy against state-vector method'
print('HEA circuit energy: ', circuit_energy(params, MF[0], inte[0], backend, cir,  xstrings, tableb, groups, signs, B))

[AerSimulator('aer_simulator'), AerSimulator('aer_simulator_statevector'), AerSimulator('aer_simulator_density_matrix'), AerSimulator('aer_simulator_stabilizer'), AerSimulator('aer_simulator_matrix_product_state'), AerSimulator('aer_simulator_extended_stabilizer'), AerSimulator('aer_simulator_unitary'), AerSimulator('aer_simulator_superop'), QasmSimulator('qasm_simulator'), StatevectorSimulator('statevector_simulator'), UnitarySimulator('unitary_simulator')]
           ┌───┐    ┌───────────┐
hea_0: ────┤ X ├────┤ Ry(gg[0]) ├
           ├───┤    ├───────────┤
hea_1: ────┤ X ├────┤ Ry(gg[1]) ├
       ┌───┴───┴───┐└───────────┘
hea_2: ┤ Ry(gg[2]) ├─────────────
       ├───────────┤             
hea_3: ┤ Ry(gg[3]) ├─────────────
       ├───────────┤             
hea_4: ┤ Ry(gg[4]) ├─────────────
       ├───────────┤             
hea_5: ┤ Ry(gg[5]) ├─────────────
       └───────────┘             


/tmp/ipykernel_185541/869746353.py:2: DeprecationWarning: The class ``qiskit.providers.provider.Provider`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  backend = AerProvider().backends()[1]
/tmp/ipykernel_185541/869746353.py:3: DeprecationWarning: The class ``qiskit.providers.provider.Provider`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  print(AerProvider().backends())


converged SCF energy = -7.77087366922191
sampled HF energy:  -7.771019774194969
           ┌───┐    ┌───────────┐           ░ ┌───────────┐            ░ »
hea_0: ────┤ X ├────┤ Ry(gg[0]) ├──■────────░─┤ Ry(gg[6]) ├───■────────░─»
           ├───┤    ├───────────┤┌─┴─┐      ░ ├───────────┤ ┌─┴─┐      ░ »
hea_1: ────┤ X ├────┤ Ry(gg[1]) ├┤ X ├──■───░─┤ Ry(gg[7]) ├─┤ X ├──■───░─»
       ┌───┴───┴───┐└───────────┘└───┘┌─┴─┐ ░ ├───────────┤ └───┘┌─┴─┐ ░ »
hea_2: ┤ Ry(gg[2]) ├──────■───────────┤ X ├─░─┤ Ry(gg[8]) ├───■──┤ X ├─░─»
       ├───────────┤    ┌─┴─┐         └───┘ ░ ├───────────┤ ┌─┴─┐└───┘ ░ »
hea_3: ┤ Ry(gg[3]) ├────┤ X ├──────■────────░─┤ Ry(gg[9]) ├─┤ X ├──■───░─»
       ├───────────┤    └───┘    ┌─┴─┐      ░ ├───────────┴┐└───┘┌─┴─┐ ░ »
hea_4: ┤ Ry(gg[4]) ├──────■──────┤ X ├──────░─┤ Ry(gg[10]) ├──■──┤ X ├─░─»
       ├───────────┤    ┌─┴─┐    └───┘      ░ ├────────────┤┌─┴─┐└───┘ ░ »
hea_5: ┤ Ry(gg[5]) ├────┤ X ├───────────────░─┤ Ry(gg[11]) ├┤ X ├──────░─»
       └───────────┘

sampled HF energy:  -7.7711428789172
           ┌───┐    ┌───────────┐           ░ ┌───────────┐            ░ »
hea_0: ────┤ X ├────┤ Ry(gg[0]) ├──■────────░─┤ Ry(gg[6]) ├───■────────░─»
           ├───┤    ├───────────┤┌─┴─┐      ░ ├───────────┤ ┌─┴─┐      ░ »
hea_1: ────┤ X ├────┤ Ry(gg[1]) ├┤ X ├──■───░─┤ Ry(gg[7]) ├─┤ X ├──■───░─»
       ┌───┴───┴───┐└───────────┘└───┘┌─┴─┐ ░ ├───────────┤ └───┘┌─┴─┐ ░ »
hea_2: ┤ Ry(gg[2]) ├──────■───────────┤ X ├─░─┤ Ry(gg[8]) ├───■──┤ X ├─░─»
       ├───────────┤    ┌─┴─┐         └───┘ ░ ├───────────┤ ┌─┴─┐└───┘ ░ »
hea_3: ┤ Ry(gg[3]) ├────┤ X ├──────■────────░─┤ Ry(gg[9]) ├─┤ X ├──■───░─»
       ├───────────┤    └───┘    ┌─┴─┐      ░ ├───────────┴┐└───┘┌─┴─┐ ░ »
hea_4: ┤ Ry(gg[4]) ├──────■──────┤ X ├──────░─┤ Ry(gg[10]) ├──■──┤ X ├─░─»
       ├───────────┤    ┌─┴─┐    └───┘      ░ ├────────────┤┌─┴─┐└───┘ ░ »
hea_5: ┤ Ry(gg[5]) ├────┤ X ├───────────────░─┤ Ry(gg[11]) ├┤ X ├──────░─»
       └───────────┘    └───┘               ░ └────────────┘└──

converged SCF energy = -7.7708736692219


HEA circuit energy:  -7.438996248680604


To test that the `circuit_energy` function works, we further benchmark it against state_vector energy without sampling, `Simple_Vec`. `Simple_Vec` gives the correct numerical energy via matrix contraction. As we can see, the `circuit_energy` 

In [6]:
def to2(num, M):

    s = []
    while num:
        s.append(num % 2)
        num = num // 2

    additional_zeros = [0] * (M - len(s))
    m = np.array(s + additional_zeros, dtype = int)
    #m = ''.join(np.array(s + additional_zeros, dtype = str))
    return m

def Simple_Vec(params, mf, core, circuit, table, form, mat):
    '''
    Vector energy: You will need one table for decoding, and one table for mapping to the matrix format

    In:
    ---
    table: send compressed strings to uncompressed ones

    form: send uncompressed string to the matrix coordinate in the subspace Hamiltonian.
    '''

    circuit = circuit.assign_parameters(params)
    
    circuit.save_density_matrix(label="rho", conditional=True)
    
    circuit = transpile(circuit, backend)
    job = backend.run(circuit)
    result = job.result()

    rho = result.data()['rho'][''].data
    basis = []
    for i in range(len(rho)):
        key = ''.join(np.array(to2(i, circuit.num_qubits), dtype = str))
        basis.append(form[fed.str_decode(key, table)])

    newrho = np.zeros((mat.shape[0], mat.shape[0]))
    for index, i in enumerate(basis):
        for jndex, j in enumerate(basis):
            newrho[i][j] = rho[index][jndex].real

    something = newrho@mat
    return np.trace(something)+core+mf.energy_nuc()


#get linearly compressed Hamiltonian in matrix form 


#the following is to map binary basis to the integer indices of mat.
mat_basis = tableb.values()

opt_basis = []
for i in mat_basis:
    opt_basis.append(np.array(list(i), dtype = int))
    
#define a second dictionary.
stuff = list(zip(np.arange(len(mat_basis)), list(mat_basis)))
form = dict()
for (a, b) in stuff:
    form[b] = a

mat = d.first_quantised(MF[0].mol, one, two, opt_basis)
print('Exact HEA numerical energy: ', Simple_Vec(params, MF[0], core, cir, tableb, form, mat))

Exact HEA numerical energy:  -7.31509250516266


## $\text{H}_2\text{O}$ demo:

Next, we provide a further demo on the $12$ modes water molecule compressed to $11$ qubits. In this experiment, while theoretically we could expect that the measuremnet groups to be upper bounded by $\binom{12}{4}+\binom{12}{2}+1 =562$, in practice we only need to prepare $212$ measurement groups for one measurement of the water molecule energy, because vast majority of the one and two integrals values are zero.

In [7]:
'H2O example'

dist = np.array([0.9584])
atoms = ['H', 'O', 'H']
Rs = np.array([1, 0, 1])
thetas = [0, 0, np.pi*104.5/180]
basis = 'sto-3g'
mult = 1

'approximation: need to test the freezing.'
freeze = 1

mols = []
for i in dist:
    R = i*Rs
    mols.append(d.plane_molecule(atoms, R, thetas, basis, mult))

M = mols[0].nao*2-freeze*2
e = mols[0].nelectron-2*freeze
print(M, e)

Q = 11

hardcore = 0


M = mols[0].nao*2-freeze*2
N = mols[0].nelectron-2*freeze

    
orb_choice = np.arange(freeze, M//2+freeze)

    
B = r.encoder(M, N, Q)
tableb = fed.look_up_parity(M, M, N, B)

#Ground State Energy -- optional
basis = d.new_hot(mols[0], orb_choice[0], orb_choice, hardcore)
functions = [[d.compute_integrals, d.get_active_space_integrals, basis]]

#get energy, ground state and the optional meanfield object.
ene, gs, MF= d.spectrum(mols, functions, orb_choice[0], orb_choice)
print('gs: ', ene[0][0])


12 8
0 0
Injectivity:  495 495 1.0
converged SCF energy = -74.9631043332761
gs:  -75.0126577040684


gs:  -75.01265770406839


In [8]:
'compute integrals and PySCF meanfield object'
integrals, MF= d.integrals(mols, functions, orb_choice)
naos = np.arange(mols[0].nao)

base = fed.Indices(integrals[0], threshold = 10**-12)
compute = fed.RDM_grouper(base)
groups = fed.measurement_group(compute)

#The number of distinct measurements required for the experiment.
print('The number of measurement basis: ', len(groups))

'compute x-string'
ustrings = fed.xfunc(groups, np.identity(M))
signs = fed.glob_sign(groups, M)

hists = fed.hist_format(gs[0][0], basis, B, ustrings)

'From the histogram, we calculate the rdm values'
da, ddaa, Vals= fed.rdm_val(hists, tableb, groups, signs, B, hardcore)

stuff, MF= d.integrals(mols, functions, orb_choice)

one = stuff[0][0]
two = stuff[0][1]
core = stuff[0][2]

print('from partial RDM: ', np.tensordot(one, da, axes=((0,1), (0,1)))+1/2*np.tensordot(two, ddaa, axes=((0,1,2,3), (0,1,2,3)))+MF[0].energy_nuc()+core)


converged SCF energy = -74.9631043332761
The number of measurement basis:  212
converged SCF energy = -74.9631043332761
from partial RDM:  -75.01256329894832


converged SCF energy = -74.9631043332761


from partial RDM:  -75.01256329894835


In [9]:
'Double check another one.'
ada = d.RDM1(gs[0][0], basis)
ddaa = d.RDM2(gs[0][0], basis)


'need to compute the integrals.'
stuff, MF = d.integrals(mols, functions, orb_choice)

one = stuff[0][0]
two = stuff[0][1]
core = stuff[0][2]


print('from RDM: ', np.tensordot(one, ada, axes=((0,1), (0,1)))+1/2*np.tensordot(two, ddaa, axes=((0,1,2,3), (0,1,2,3)))+MF[0].energy_nuc()+core)

converged SCF energy = -74.9631043332761
from RDM:  -75.01265770406837


In [10]:
'Test H2O rdm'
for i in zip(groups, Vals):
    for index, j in enumerate(i[0]):
        if len(j) == 2:
            
            if np.abs((np.abs(ada[j[0], j[1]])-np.abs(i[1][index])))>1e-15:
                print('1-RDM: ', j, ada[j[0], j[1]], i[1][index])
                
        if len(j) == 4:
            if np.abs((np.abs(ddaa[j[0], j[1], j[2], j[3]])-np.abs(i[1][index])))>1e-10:
                print('2-RDM: ', j, ddaa[j[0], j[1], j[2], j[3]], i[1][index])

## Conclusion:

We demonstrated our RLE-FED encoding-decoding protocol on the $\text{LiH}$ molecule. We explicitly show how to make use of a decoder, in this instance, a look-up-table `tableb` to avoid qubit operator decomposition in the encoded space. Resorting to the FED algorithm, we can decode the histograms `hists` from quantum computer before calculating the expectation values. We benchmark the FED algorithm and shows that it can correctly determine the energy of ground state and a randomly parameterized quantum circuit. 

We also developed the cost function `circuit_energy` that samples, decode and compute the energy from a parameterized quantum circuit (PQC). To sample, the subroutine `clifford_basis` is required to prepare the PQC in different clifford basis corresponding to different RDM observables. Once the set of histograms are obtained, the rest follows. We benchmark the cost function against statevector (density matrix) simulation and show that the energy matches.

Finally, we provide a demo on the $11$ qubits water molecule to give a sense of how the measurement basis `groups` scales with the number of qubits. 